In [1]:
import re
import sys
from collections import namedtuple
Point = namedtuple('Point', 'x y')

In [2]:
pairs = {}

fn = 'input.txt'
with open(fn) as f:
    lines = f.read().strip().split('\n')
    
    min_x = sys.maxsize
    min_y = sys.maxsize
    max_x = 0
    max_y = 0
    
    for l in lines:
        x1, y1, x2, y2 = list(map(int, re.findall(r'-?\d+', l)))
        min_x = min(min_x, x1, x2)
        max_x = max(max_x, x1, x2)
        min_y = min(min_y, y1, y2)
        max_y = max(max_y, y1, y2)
        pairs[Point(x1, y1)] = Point(x2, y2)

In [3]:
width = max_x - min_x + 1
height = max_y + 1

print(width, 'x', height, '=', width*height)

4814447 x 4959119 = 23875415592193


In [4]:
def line_intersection(line1, line2):
    """ https://www.arndt-bruenner.de/mathe/9/geradedurchzweipunkte.htm
    https://stackoverflow.com/a/20677983/723769
    """
    xdiff = (line1[0].x - line1[1].x, line2[0].x - line2[1].x)
    ydiff = (line1[0].y - line1[1].y, line2[0].y - line2[1].y)

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        raise Exception('lines do not intersect')

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return Point(x, y)

y_test= 10
if (fn == 'input.txt'):
    y_test = 2000000
y_line = [Point(min_x, y_test), Point(max_x, y_test)]
covered = []
for sensor, beacon in pairs.items():
    
    # Manhattan distance
    d = abs(sensor.x - beacon.x) + abs(sensor.y - beacon.y)
    
    # trbl points for area covered by sensor (Rhombus)
    top    = Point(sensor.x,     sensor.y - d)
    right  = Point(sensor.x + d, sensor.y)
    bottom = Point(sensor.x,     sensor.y + d)
    left   = Point(sensor.x - d, sensor.y)
    
    if (top.y <= y_test <= sensor.y):
        # intersection is in upper half
        i1 = line_intersection([left, top], y_line)
        i2 = line_intersection([right, top], y_line)
        covered.append(sorted([i1.x, i2.x]))
    elif (bottom.y >= y_test > sensor.y):
        # intersection is in lower half
        i1 = line_intersection([left, bottom], y_line)
        i2 = line_intersection([right, bottom], y_line)
        covered.append(sorted([i1.x, i2.x]))
    else:
        # not intersection with current sensor/beacon
        pass

covered = sorted(covered)

# why can we assume only to use the min and max?
# why ist guaranteed the the covered line is continous?!
xs_min = []
xs_max = []
for i in covered:
    xs_min.append(i[0])
    xs_max.append(i[1])
    
minx = min(xs_min)
maxx = max(xs_max)

print(abs(minx) + maxx)

5716881.0


# Part 2

In [5]:
from intervaltree import Interval, IntervalTree
t = IntervalTree()

maxxy = 20
if (fn == 'input.txt'):
    maxxy = 4000000

for y in range(2000000, maxxy+1):
    
    y_test= y
    y_line = [Point(min_x, y_test), Point(max_x, y_test)]
    covered = []
    edges = []
    for sensor, beacon in pairs.items():
        d = abs(sensor.x - beacon.x) + abs(sensor.y - beacon.y)

        top    = Point(sensor.x,     sensor.y - d)
        right  = Point(sensor.x + d, sensor.y)
        bottom = Point(sensor.x,     sensor.y + d)
        left   = Point(sensor.x - d, sensor.y)
        
        if (top.y <= y_test <= sensor.y):
            # intersection is upper half
            i1 = line_intersection([left, top], y_line)
            i2 = line_intersection([right, top], y_line)
            
            covered.append(sorted([i1.x, i2.x]))
            edges.append(i1.x-1)
            edges.append(i2.x+1)
        elif (bottom.y >= y_test > sensor.y):
            # intersection in lower half
            i1 = line_intersection([left, bottom], y_line)
            i2 = line_intersection([right, bottom], y_line)
    
            covered.append(sorted([i1.x, i2.x]))
            edges.append(i1.x-1)
            edges.append(i2.x+1)
        else:
            # not intersection with current sensor/beacon
            pass

    covered = sorted(covered)
    
    # to reduce the size of the for loop
    # we collected the edges outside of each range
    # now we only need to check if the edges are inside
    # other ranges.
    for x in edges:
        if not (0 <= x <= maxxy):
            continue
        
        uncovered = True
        for c in covered:
            if c[0] <= x <= c[1]:
                uncovered = False
        if (uncovered):
            print(4000000*x+y)
            break
            y = maxxy # abort outer loop, only one solution is possible
    

10852583132904.0
